In [ ]:
import json
import os

import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None, "GOOGLE_API_KEY is not set"

ROUTE_OPTIONS = {
    "M5": {
        "latitude": 51.857149,
        "longitude": -2.177166,
    },
    "A40": {
        "latitude": 51.867005,
        "longitude": -2.196442,
    },
}

In [ ]:
routes = {}
for route_name, route_coords in ROUTE_OPTIONS.items():
    resp = requests.post(
        "https://routes.googleapis.com/directions/v2:computeRoutes",
        json={
            "origin": {
                "location": {
                    "latLng": {
                        "latitude": 51.847289,
                        "longitude": -2.174274,
                    },
                },
            },
            "destination": {
                "location": {
                    "latLng": {
                        "latitude": 51.894156,
                        "longitude": -2.131170,
                    },
                },
            },
            "intermediates": [
                {
                    "location": {
                        "latLng": {
                            "latitude": route_coords["latitude"],
                            "longitude": route_coords["longitude"],
                        },
                    },
                },
            ],
            "travelMode": "DRIVE",
            "extraComputations": ["TRAFFIC_ON_POLYLINE"],
            "routingPreference": "TRAFFIC_AWARE_OPTIMAL"
        },
        headers={
            "X-Goog-Api-Key": GOOGLE_API_KEY,
            "X-Goog-FieldMask": ",".join([
                "routes.duration",
                "routes.distanceMeters",
                "routes.polyline",
                "routes.travelAdvisory"
            ])
        },
    )
    
    route = resp.json()["routes"][0]
    routes[route_name] = route

In [ ]:
import numpy as np

In [ ]:
from shapely.geometry import LineString
from scipy.interpolate import splprep, splev

In [ ]:
def polyline_decode(encoded_polyline):
    """
    Decode an encoded polyline string into a list of (latitude, longitude) tuples.

    Args:
        encoded_polyline: A string containing the encoded polyline

    Returns:
        A list of (lat, lng) tuples
    """
    def _decode_value(encoded, index):
        """
        Decode a single signed integer value from the encoded polyline.

        Args:
            encoded: The encoded polyline string
            index: Current position in the string

        Returns:
            Tuple of (decoded_value, new_index)
        """
        result = 0
        shift = 0
        byte = 0

        while True:
            # Get ASCII value and subtract 63
            byte = ord(encoded[index]) - 63
            index += 1

            # Extract the 5-bit chunk (bits 0-4)
            chunk = byte & 0x1F

            # Add to result
            result |= (chunk << shift)

            # Check if more chunks follow (bit 5 is set)
            if (byte & 0x20) == 0:
                break

            shift += 5

        # Check if the value is negative (bit 0 of result is set)
        if result & 1:
            # Negative value: invert and add 1 (two's complement)
            result = ~result
            result = result & 0xFFFFFFFF  # Keep it as 32-bit
            result = (result >> 1) | 0x80000000  # Right-shift and set sign bit
        else:
            # Positive value: just right-shift
            result = result >> 1

        # Convert to signed 32-bit integer
        if result & 0x80000000:
            result = result - 0x100000000

        return result, index
    if not encoded_polyline:
        return []

    points = []
    index = 0
    lat = 0
    lng = 0

    while index < len(encoded_polyline):
        # Decode latitude
        lat_change, index = _decode_value(encoded_polyline, index)
        lat += lat_change

        # Decode longitude
        lng_change, index = _decode_value(encoded_polyline, index)
        lng += lng_change

        points.append((lat / 1e5, lng / 1e5))

    return points

In [ ]:
for route_name, route in routes.items():
    coords = np.array(polyline_decode(route["polyline"]["encodedPolyline"]))
    line = LineString(coords)
    simple = line.simplify(tolerance=0.001, preserve_topology=False)
    
    # fit spline
    tck, _ = splprep([simple.xy[0], simple.xy[1]], s=0)
    
    # sample smoothly
    u = np.linspace(0, 1, 300)
    x_smooth, y_smooth = splev(u, tck)
    
    plt.plot(y_smooth, x_smooth, label=route_name)
    plt.legend()
    plt.axis("off")
    plt.show()